In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import plot, show, savefig, xlim, figure,  ylim, legend, boxplot, setp, axes

from sklearn.cluster import Birch
from sklearn import metrics
from sklearn.metrics import pairwise_distances

from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
df =  pd.read_csv('raw_data.csv')
df.columns =["start", "total", 'overhead', 'duration']
print(df.describe())
df['c'] = df['total'].diff()
df_filtered = df[df['c'] != 0]

index_list = df_filtered.index.tolist() # list of the start poisitions of index for change of values 
# print(len(index_list))
# print('\n')
# print(index_list)

l_mod = index_list + [max(index_list)+1] # creating a list of indexes to iterate over (must have 0 in it)
list_of_dfs = [df.iloc[l_mod[n]:l_mod[n+1]] for n in range(len(l_mod)-1)] # creating a list of dfs for each index

# len(list_of_dfs)

              start         total      overhead       duration
count  56706.000000  56706.000000  56706.000000   56706.000000
mean     486.235319    490.679999      1.977623     483.287941
std       96.924995     98.011656      1.502224    4894.567496
min      194.000000    200.000000      0.817215       2.000000
25%      399.000000    400.000000      1.370191     172.000000
50%      497.000000    500.000000      1.742813     297.000000
75%      591.000000    600.000000      2.214598     454.000000
max      599.000000    600.000000     30.000000  230964.000000


In [4]:
data = pd.DataFrame(columns=['median', 'q1', 'q3', 'p90', 'cars'])
mode = []
median, q1, q3, p90 = [], [], [], []

for frame in list_of_dfs:
    m = frame['overhead'].median()
    median.append(m)
    q1.append(frame['overhead'].quantile(q =.25))
    q3.append(frame['overhead'].quantile(q =.75))
    p90.append(frame['overhead'].quantile(q =.9))
    mode.append(frame['total'].mode()[0]) # mode returns a series so we need to specify which value to return

    
data['median'] = median
data['q1'] = q1
data['q3'] = q3
data['p90']= p90
data['cars'] = mode

data.head()

,median,q1,q3,p90,cars
0,1.570335,1.352734,1.911584,2.235225,200
1,1.598932,1.317073,1.924391,2.327385,300
2,1.710585,1.378743,2.100281,2.602389,400
3,1.619190,1.350614,1.929671,2.363173,300
4,1.521436,1.297669,1.800330,2.134260,200


In [6]:
data.shape

(51, 5)

In [7]:
data.drop([50], inplace = True)

In [8]:
data.shape

(50, 5)

In [13]:
data_split = data[['median', 'q1', 'q3', 'p90', 'cars']].to_numpy()
data_split = np.split(data_split, 5)

In [14]:
data_split_z = data[['median', 'q1', 'q3', 'p90', 'cars']].to_numpy()
# print_graphs(data_split_z, 'Before outlier detection')
data_split_z = np.split(data_split_z, 10)
indexes = [] 
for d in range(len(data_split_z)):
    ind = []
    k = len(data_split_z[d])
    for i in [0, 1, 2, 3]:
        ys = data_split_z[d][:,i] 
        mean_ys = np.mean(ys)
        std_ys = np.std(ys)
        
        for y in range(len(data_split_z[d])):
            z_score = (data_split_z[d][y, i]  - mean_ys) / std_ys
            if np.abs(z_score) > 3:
                ind.append(y)
                
    data_split_z[d] = np.delete(data_split_z[d], ind, axis=0)
    print('Outliers detected: ' + str((k - len(data_split_z[d]))))
#     print('\n')


print(indexes)
data_split_z = np.concatenate(data_split_z, axis=0)
# print_graphs(data_split_z, 'After outlier detection')

Outliers detected: 0
Outliers detected: 0
Outliers detected: 0
Outliers detected: 0
Outliers detected: 0
Outliers detected: 0
Outliers detected: 0
Outliers detected: 0
Outliers detected: 0
Outliers detected: 0
[]


In [18]:
data_split_modified_z = data[['median', 'q1', 'q3', 'p90', 'cars']].to_numpy()
indexes = [] 
# print_graphs(data_split_modified_z, 'Before outlier detection')
data_split_modified_z = np.split(data_split_modified_z, 10)

for d in range(len(data_split_modified_z)):
    print(d)
    ind = []
    k = len(data_split_modified_z[d])
    for i in [0, 1, 2, 3]:
        ys = data_split_modified_z[d][:,i] 
        median_y = np.median(ys)
        median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in ys])
#         print(median_absolute_deviation_y)
        for y in range(len(data_split_modified_z[d])):
            modified_z_score = 0.6745 * (data_split_modified_z[d][y, i] - median_y) / median_absolute_deviation_y
#             z_score = (data_split[d][y, i]  - mean_ys) / std_ys
            if np.abs(modified_z_score) > 3.5:
                ind.append(y)


    data_split_modified_z[d] = np.delete(data_split_modified_z[d], ind, axis=0)
    print('Outliers detected: ' + str((k - len(data_split_modified_z[d]))))
    for num in range(len(ind)):
        j = ind[num] + (d*5)
        ind[num] = j
        
    indexes.append(ind)
#     print('\n')
print(indexes)
data_split_modified_z = np.concatenate(data_split_modified_z, axis=0)
# print_graphs(data_split_modified_z, 'After outlier detection')

0
Outliers detected: 2
1
Outliers detected: 1
2
Outliers detected: 0
3
Outliers detected: 0
4
Outliers detected: 2
5
Outliers detected: 1
6
Outliers detected: 1
7
Outliers detected: 2
8
Outliers detected: 1
9
Outliers detected: 0
[[2, 4], [9, 9], [], [], [20, 22], [26], [30], [36, 38], [40], []]


In [ ]:
o = [2,4, 19, 60, 62, 76, 90, 106, ]